In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [6]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
mint_filt = Filter.create_filter(address=None, event_types=[Pair.events.Mint])  # Listen events from any smart contract

In [35]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [36]:
start = start - 100

In [37]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=mint_filt)

In [38]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['buyer'] = Conversion().convert_uint256_hex_string_to_address(topics[1])

    if(len(arguments) >= 1):
        event['details']['purchase_price'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,311,766 tx:0x34918614379aace0df96fa02149c59ba5fad24e892356af6477f0956cbf9364e
Transfer at block:60,311,783 tx:0xc0ea0b3d8824274c230d820694cfb267b24d9f844505ca1c553aa3865c928f8c
Transfer at block:60,311,815 tx:0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a9a3ecc26b003fb8be551
Transfer at block:60,311,815 tx:0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a9a3ecc26b003fb8be551
Transfer at block:60,311,815 tx:0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a9a3ecc26b003fb8be551
Transfer at block:60,311,832 tx:0xf89fb221f092107946248ac96f86660a3f43ee74eaf2a4a5b635bfe59948f2e7
Transfer at block:60,311,832 tx:0xf89fb221f092107946248ac96f86660a3f43ee74eaf2a4a5b635bfe59948f2e7
Transfer at block:60,311,832 tx:0xf89fb221f092107946248ac96f86660a3f43ee74eaf2a4a5b635bfe59948f2e7
Transfer at block:60,311,832 tx:0xf89fb221f092107946248ac96f86660a3f43ee74eaf2a4a5b635bfe59948f2e7
Transfer at block:60,311,833 tx:0x47019165a57159c7fb77786b29a875c3e9e6b83eaaffd07e13a2c93a3218d4a5
Transfer a

In [39]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0xf2b55154f9071a33981e92c79c049b89b7fc90de',
  'tx_hash': '0x34918614379aace0df96fa02149c59ba5fad24e892356af6477f0956cbf9364e',
  'blk_num': 60311766,
  'timestamp': 1723059972,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0x8771ee85d21313a90151C34a4CC919Cc7371BF31',
   'purchase_price': 1723059972}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x9e621dcf9e5205d7ee5fbfdda6ae3a6a2de2a776',
  'tx_hash': '0xc0ea0b3d8824274c230d820694cfb267b24d9f844505ca1c553aa3865c928f8c',
  'blk_num': 60311783,
  'timestamp': 1723060008,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0x8771ee85d21313a90151C34a4CC919Cc7371BF31',
   'purchase_price': 1723060008}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x27f8d03b3a219

In [31]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,mint,sushi,0x9e621dcf9e5205d7ee5fbfdda6ae3a6a2de2a776,0xc4c03fbf594f28a65fefeccf0668f3719db624ac98ec...,60311732,1723059900,{'web3_type': {'address': '0x9e621dcf9e5205d7e...
1,polygon,uniswapv2pair,mint,sushi,0x9e621dcf9e5205d7ee5fbfdda6ae3a6a2de2a776,0xb5568147c43879dafd25d1c21f647a491c38caff7a75...,60311744,1723059926,{'web3_type': {'address': '0x9e621dcf9e5205d7e...
2,polygon,uniswapv2pair,mint,sushi,0x9d083c192c44fac3e07786d787918c456f58a27c,0xf4dc2afe5da22490903bba8b621a75efc54591de1be1...,60311747,1723059932,{'web3_type': {'address': '0x9d083c192c44fac3e...
3,polygon,uniswapv2pair,mint,sushi,0xf2b55154f9071a33981e92c79c049b89b7fc90de,0x34918614379aace0df96fa02149c59ba5fad24e89235...,60311766,1723059972,{'web3_type': {'address': '0xf2b55154f9071a339...
4,polygon,uniswapv2pair,mint,sushi,0x9e621dcf9e5205d7ee5fbfdda6ae3a6a2de2a776,0xc0ea0b3d8824274c230d820694cfb267b24d9f844505...,60311783,1723060008,{'web3_type': {'address': '0x9e621dcf9e5205d7e...
5,polygon,uniswapv2pair,mint,sushi,0x27f8d03b3a2196956ed754badc28d73be8830a6e,0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a...,60311815,1723060076,{'web3_type': {'address': '0x27f8d03b3a2196956...
6,polygon,uniswapv2pair,mint,sushi,0x27f8d03b3a2196956ed754badc28d73be8830a6e,0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a...,60311815,1723060076,{'web3_type': {'address': '0x27f8d03b3a2196956...
7,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x43a94ac7b8003c8f153e41a1e8b7ff20896c94e45e6a...,60311815,1723060076,{'web3_type': {'address': '0x1a13f4ca1d028320a...
